In [10]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

wttj_url = 'https://www.welcometothejungle.com/fr/companies/hunteed/jobs/community-success-manager-community-builder_paris?q=6d2ebe57905edc93351a1b0e81bdf060&o=802986be-3cbd-4743-8bd2-b6830b8a75a2'
page = requests.get(wttj_url)
page.text 

soup = bs(page.text, "html.parser")
#if page.status_code == 200:
    #print(f"reussi de la requête. Code de statut: {page.status_code}")
#else:
   #@ print(f"Échec de la requête. Code de statut: {page.status_code}")
#soup

In [2]:
def get_info(link):

    page = requests.get(link)
    soup = bs(page.text, "html.parser")
    company = soup.find('span', attrs = {'class' : "sc-empnci hmOCpj wui-text"}).string
    job = soup.find('h2', attrs = {'class': "sc-empnci cYPTxs wui-text"}).string
    lieu = soup.find('span', attrs = {'class': "sc-1eoldvz-2 kGQqJy"}).string
    spans = soup.find_all('span', attrs = {'class': "sc-bXCLTC kJcLKT"})
    try:
        if spans:
            start = spans[1].find_next("span").string
    except:
        start = "Date de prise de poste non trouvée"
    try : 
        if soup.find("time").has_attr('datetime'):
            debut= soup.find("time").string
    except : 
            debut = "Non trouvé"
        
    col = soup.find_all('span', attrs = {'class': "sc-empnci klToKr wui-text"})
    domaine = col[0].find_next("span").string
    line = [company,job,lieu,domaine,debut,link]
    return line
get_info('https://www.welcometothejungle.com/fr/companies/hunteed/jobs/community-success-manager-community-builder_paris?q=6d2ebe57905edc93351a1b0e81bdf060&o=802986be-3cbd-4743-8bd2-b6830b8a75a2')

['Hunteed',
 'Customer Success Manager B2C H/F',
 'Paris',
 'SaaS / Cloud Services, Recrutement, Digital',
 'il y a 8 heures',
 'https://www.welcometothejungle.com/fr/companies/hunteed/jobs/community-success-manager-community-builder_paris?q=6d2ebe57905edc93351a1b0e81bdf060&o=802986be-3cbd-4743-8bd2-b6830b8a75a2']

In [3]:
tous = soup.find_all('span')

#line=[company,job,lieu,contrat,debut,domaine,taille]
#return line 

In [4]:
url2 = 'https://www.welcometothejungle.com/fr/companies/favikon/jobs/charge-de-marketing-communication-h-f_paris?q=09514f03310bf102a5f7e96b624cd02e&o=b4a4fa03-9df7-48a5-b038-d98148553c03'

In [5]:
get_info(url2)

['Favikon',
 'Assistant(e) de Marketing & Communication H/F - Stage',
 'Paris',
 'AdTech  / MarTech',
 'hier',
 'https://www.welcometothejungle.com/fr/companies/favikon/jobs/charge-de-marketing-communication-h-f_paris?q=09514f03310bf102a5f7e96b624cd02e&o=b4a4fa03-9df7-48a5-b038-d98148553c03']

In [6]:
jobs_list = []
page_number = 1

for i in range(1,35):
    all_jobs = f"https://www.welcometothejungle.com/fr/jobs?query=&page={page_number}&aroundQuery=worldwide"
    page_number +=1
    page = requests.get(all_jobs)
    soup = bs(page.text,'html.parser')
    divs = soup.find_all('div',class_="sc-bXCLTC sc-emIrwa bsJILN sc-6i2fyx-4 bpkkNp")
    for div in divs : 
        a = div.find("a")
        if ("/fr/companies" in a["href"]) and (a["href"] not in jobs_list):
            jobs_list.append(a["href"])
            
            
            
jobs_list = list(set(jobs_list)) # évite les doublons
#len(jobs_list)

print(len(jobs_list))

0


In [29]:
base = "https://www.welcometothejungle.com"
df = pd.DataFrame(columns=["Company","Job","Lieu","Domaine","Mise en ligne","Link"])

for job in jobs_list : 
    #print("Ofrre ",i)
    row = get_info(base+job)
    a = pd.DataFrame([row],columns=["Company","Job","Lieu","Domaine","Mise en ligne","Link"])
    df = pd.concat([df,a],ignore_index=True)
    
    #print("---------------------------")
df = df.drop_duplicates().reset_index(drop=True)
df

,Company,Job,Lieu,Domaine,Mise en ligne,Link
0,Altice France SFR RMC BFM,[SFR] Responsable Communication (h/f) - Commun...,Paris,"Média, Télévision / Production audiovisuelle, ...",il y a 17 heures,https://www.welcometothejungle.com/fr/companie...
1,Thales,Responsable d'offres - Systèmes de communicati...,Gennevilliers,"Logiciels, Cybersécurité, Aéronautique / Spatiale",il y a 16 heures,https://www.welcometothejungle.com/fr/companie...
2,Thales,Chef de Projet Produit (PPM) Communications Aé...,Gennevilliers,"Logiciels, Cybersécurité, Aéronautique / Spatiale",il y a 16 heures,https://www.welcometothejungle.com/fr/companie...
3,Decathlon,STAGE Communication Atelier régional (platefor...,Mondeville,"Sport, Distribution sélective",il y a 17 heures,https://www.welcometothejungle.com/fr/companie...
4,Thales,Responsable d'offres - Systèmes de communicati...,Gennevilliers,"Logiciels, Cybersécurité, Aéronautique / Spatiale",il y a 16 heures,https://www.welcometothejungle.com/fr/companie...
5,Thales,Ingénieur IVVQ - Unités de Communication - F/H,Cholet,"Logiciels, Cybersécurité, Aéronautique / Spatiale",il y a 16 heures,https://www.welcometothejungle.com/fr/companie...


In [55]:
wttj_url = 'https://www.welcometothejungle.com/fr/jobs?query=&page=1&aroundQuery=worldwide'
page = requests.get(wttj_url)
page.text 
soup = bs(page.text, "html.parser")
tous = soup.find_all('div')


all_elements_with_classes = soup.find_all('div',{'data-objectid':True})

# Initialiser un dictionnaire pour stocker le compte de chaque classe
class_count = {}

# Compter le nombre d'occurrences de chaque classe
for element in all_elements_with_classes:
    classes = element.get('class', [])
    for class_name in classes:
        class_count[class_name] = class_count.get(class_name, 0) + 1

# Afficher le compte de chaque classe
for class_name, count in class_count.items():
    (f"La classe '{class_name}' apparaît {count} fois.")

all_elements_with_classes


[]

In [40]:
df.to_csv(r'./welcome_to_the¨jungle.csv', index=False, header=True)

In [44]:
div_element = soup.find('div')

# Extraire la liste des classes qui contiennent 'sc-bXCLTC'
if div_element:
    classes_matching = [classe for classe in div_element.get('class', []) if 'sc-bXCLTC' in classe]
    print(classes_matching)
else:
    print("Élément non trouvé")

[]
